## NEvent Data Post-Processing


In [1]:
# First, load the necessary packages
import os
import numpy as np
import glob as glob
import mne
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import scipy as sp
import scipy.stats as spst
import pandas as pd
import sys
import time
import joblib

# Determine which computer you're running on, setting the /mindstore/ezzyatlab 
# server location accordingly
import socket
if socket.gethostname() == 'Youssefs-iMac.local': # home office
    server_folder = '/Volumes/ezzyatlab/'
elif socket.gethostname() == 'Youssefs-MacBook-Air.local': # laptop
    server_folder = '/Volumes/ezzyatlab/'
elif socket.gethostname() == 'yezzyat-21': # lab office
    server_folder = '/Volumes/ezzyatlab/'
else:
    server_folder = '/Volumes/ezzyatlab/'

# Using the server location, load a library of lab code to be used
# for post-processing

sys.path.append(server_folder + 'labutils/') 
sys.path.append(server_folder + 'labutils/scalpeeg/') 

#exp_folder = server_folder + 'experiments/NEvent/'

#participants = pd.read_csv(exp_folder + 'participants.tsv',
#                           delimiter='\t')

print(f'\nMNE-Python Version: {mne.__version__}\n')

#electrodes of interests (for graphing)
electrode_names = ['Cz', 'Pz', 'Fz', 'P3', 'P4', 'C4', 'C3', 'FC3']



MNE-Python Version: 1.3.0



In [7]:
#matplotlib.use('TkAgg')

In [2]:
# Set the subject code/number for the to-be-processed dataset
subject_code = 'sub-062'

# Set the path to the data folder
data_folder = server_folder + 'experiments/NEvent/exp_eeg_v1/' 
data_raw_file = os.path.join(data_folder,subject_code,
                             'eeg','raw',f'{subject_code}_NEvent-task.vhdr')


# Low and high frequency filters for raw data
raw_l_freq = 0.5
raw_h_freq = 200
linenoise_min = 60
linenoise_max = 181

# ICA parameters
ica_sfreq = 250             # Resampling frequency
ica_l_freq = 1.             # Filter cutoff: low
ica_h_freq = None           # Filter cutoff: high
ica_flat = dict(eeg=5e-6)   # Minimum channel amplitude for inclusion

# Set the channel locations montage
# montage = mne.channels.make_standard_montage('GSN-HydroCel-65_1.0')
# montage.ch_names[-1] = 'E65'

# # Rename the channels to 10-10 convention
# tenten_file = os.path.join(server_folder,'labdocs','scalp','10-10_vs_EGI.csv')
# #tenten = pd.read_excel(tenten_file)
# tenten = pd.read_csv(tenten_file)

# # Define old -> new mapping
# chan_name_map = dict(zip(tenten.Labels_EGI64,tenten.Labels_1010)) 

# Scale factors, for use in raw.plot()
scalings = dict(mag=1e-12, grad=4e-11, eeg=75e-6, eog=150e-6, ecg=5e-4,
     emg=1e-3, ref_meg=1e-12, misc=1e-3, stim=1,
     resp=1, chpi=1e-4, whitened=1e2)


In [3]:
# Load the raw data object and apply the 10-10 channel names
raw = mne.io.read_raw_brainvision(data_raw_file, preload=True)

# Apply new channels names to raw object
# raw.set_montage(montage)
# mne.rename_channels(raw.info,chan_name_map)

# # Change channel type for Cz
# raw.set_channel_types({'Cz': 'misc'})
    
# # Create a list of the data channels (excluding Cz, EOG, STIM, etc)
#data_channels = raw.ch_names[0:60]
data_channels = raw.ch_names

# Display the data, just make sure it looks like raw data
# and that the channel names imported
#fig = raw.plot(start=0, duration=60, n_channels=30, scalings=scalings)

#print(f'{raw.n_times/(1000)} seconds')

Extracting parameters from /Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/sub-062/eeg/raw/sub-062_NEvent-task.vhdr...
Setting channel info structure...
Reading 0 ... 6179659  =      0.000 ...  6179.659 secs...


## Apply minimal filtering
Filter the raw data to remove low-frequency drifts and high-frequency noise. Also apply a notch filter to remove the effects of 60 Hz line noise.


In [5]:
fig = raw.plot(start=0, duration=60, n_channels=30, scalings=scalings)

Using qt as 2D backend.
Using pyopengl with version 3.1.6


In [4]:
raw.load_data().filter(l_freq=raw_l_freq, h_freq=raw_h_freq)
raw.notch_filter(np.arange(linenoise_min,linenoise_max,60))

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 2e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 200.00 Hz
- Upper transition bandwidth: 50.00 Hz (-6 dB cutoff frequency: 225.00 Hz)
- Filter length: 6601 samples (6.601 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:   14.1s finished


Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 6601 samples (6.601 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:   13.1s finished


<RawBrainVision | sub-062_NEvent-task.eeg, 63 x 6179660 (6179.7 s), ~2.90 GB, data loaded>

## Manually annotate bad data spans

In [5]:
# Bring up a figure. Review the full timeseries for every 
# electrode and annotate spans of obvious bad data.

fig = raw.plot(start=0, duration=60, n_channels=30, scalings=scalings)


Using qt as 2D backend.
Using pyopengl with version 3.1.6
Channels marked as bad:
['F3', 'T8']


In [6]:
# Save your annotations to a csv file
raw.annotations.save(data_raw_file.
                     replace('.vhdr','-annotations.csv').replace('/raw/','/postproc/'),overwrite=True)

In [7]:
#Identify the timepoint corresponding to end of the session
# and set tmax to that value. This will crop the raw file and
# remove any post-experiment portion of the recording.
raw.info
#raw.crop(tmax=5830)
#raw.crop(tmin=170)

<Info | 9 non-empty values
 bads: 2 items (F3, T8)
 ch_names: Fp1, Fz, F3, F7, FT9, FC5, FC1, C3, T7, TP9, CP5, CP1, Pz, P3, ...
 chs: 63 EEG
 custom_ref_applied: False
 dig: 66 items (3 Cardinal, 63 EEG)
 highpass: 0.5 Hz
 lowpass: 200.0 Hz
 meas_date: 2024-03-01 19:41:59 UTC
 nchan: 63
 projs: []
 sfreq: 1000.0 Hz
>

In [8]:

# Identify bad electrodes(s)
if raw.info['bads']:
    print("Bad channels:", raw.info['bads'])

    # Find the index(es) of the bad channel(s)
    bad_channel_idxs = mne.pick_channels(raw.info['ch_names'], include=raw.info['bads'])

    # Set the bad channel(s) data to NaN
    for idx in bad_channel_idxs:
        raw._data[idx, :] = np.nan  
else:
    print("No bad channels identified.")


Bad channels: ['F3', 'T8']


## Save the filtered+annotated+cropped data to a new file 

In [9]:
fname = os.path.join(data_folder,subject_code,'eeg','postproc',
             f'{subject_code}_filtered_annot_crop.fif')

raw.load_data().save(fname,overwrite=True)

Writing /Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/sub-062/eeg/postproc/sub-062_filtered_annot_crop.fif


/var/folders/np/qpdx19z16vj6zqjjk4pnx35r0000gn/T/ipykernel_4427/2514422734.py:4: RuntimeWarning: This filename (/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/sub-062/eeg/postproc/sub-062_filtered_annot_crop.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.load_data().save(fname,overwrite=True)


Closing /Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/sub-062/eeg/postproc/sub-062_filtered_annot_crop.fif
[done]


In [70]:
#fname = os.path.join(data_folder, subject_code, 'eeg', 'postproc', 
                #     f'{subject_code}_filtered_annot_crop_2.fif')

# Read the EEG data from the file
#raw = mne.io.read_raw_fif(fname)

Opening raw data file /Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/sub-052/eeg/postproc/sub-052_filtered_annot_crop_2.fif...


/var/folders/np/qpdx19z16vj6zqjjk4pnx35r0000gn/T/ipykernel_63171/2414550595.py:5: RuntimeWarning: This filename (/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/sub-052/eeg/postproc/sub-052_filtered_annot_crop_2.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(fname)


    Range : 0 ... 5622779 =      0.000 ...  5622.779 secs
Ready.


## Create EEG epochs from event markers 

In [10]:
def load_and_process_eeg_data(data_folder, subject_code):
    # Construct the file path
    fname = os.path.join(data_folder, subject_code, 'eeg', 'postproc', 
                         f'{subject_code}_filtered_annot_crop.fif')

    # Read the EEG data from the file
    raw = mne.io.read_raw_fif(fname)

    # fig = raw.plot(start=0, duration=60, n_channels=30, scalings=scalings)

    # Get the data channels
    data_channels = raw.ch_names

    # Return the raw data and the channel names
    return raw, data_channels

#call function
raw, data_channels = load_and_process_eeg_data(data_folder, subject_code)


Opening raw data file /Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/sub-062/eeg/postproc/sub-062_filtered_annot_crop.fif...


/var/folders/np/qpdx19z16vj6zqjjk4pnx35r0000gn/T/ipykernel_4427/93895693.py:7: RuntimeWarning: This filename (/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/sub-062/eeg/postproc/sub-062_filtered_annot_crop.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(fname)


    Range : 0 ... 6179659 =      0.000 ...  6179.659 secs
Ready.


In [11]:
def read_event_markers (raw):
    # Read the event markers and display them. Confirm the 
    # codes correspond to the dictionary in the next cell
    events, tmp = mne.events_from_annotations(raw)

    # Count up the number of each event
    values, counts = np.unique(events[:,2], return_counts=True)
    print('\nEvent counts:',counts)
    #standard should be 480 480
    possible_events = {'Bit0/B  1': 'Tone',
                   'Bit1/B  1': 'Image',
                   'Bit2/B  1': 'Arrow',
                   'Bit3/B  1': 'Distance',
                   'Bit4/B  1': 'Order',
                   'Bit5/B  1': 'Source',
                   'Bit6/B  1': 'Start',
                   'New Segment/':'New Segment/'}

    event_id = {}
    for ikey in tmp.keys():
        event_id[possible_events[ikey]] = tmp[ikey]

    del tmp
    return events, counts, event_id

#call function
events, event_counts, event_id = read_event_markers(raw)


Used Annotations descriptions: ['Bit0/B  1', 'Bit1/B  1', 'Bit2/B  1', 'Bit3/B  1', 'Bit4/B  1', 'Bit5/B  1', 'New Segment/']

Event counts: [480 480  15 105 105 270   1]


In [12]:
def create_epochs(raw,events, event_id):
    
    tmin, tmax = -0.1, 1

    condition_label = 'Tone'
    # Create epochs
    epochs = mne.Epochs(raw, events, event_id=event_id[condition_label], 
                    tmin=tmin, tmax=tmax, baseline=(None,0))

    # Load the epoch data into an array
    #epoch_data = epochs.get_data(picks='Pz')
    epoch_data = epochs.get_data(picks=data_channels)

    # Convert the data into units of microvolts!
    epoch_data = epoch_data * 1e6
    return epoch_data, condition_label, epochs

epoch_data, condition_label, epochs = create_epochs(raw,events, event_id)

Not setting metadata
480 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 480 events and 1101 original time points ...
0 bad epochs dropped


## Load the behavioral data into a dataframe

In [13]:

def load_beh(server_folder, subject_code):
    beh_path = server_folder + 'experiments/NEvent/exp_eeg_v1/eeg_beh_results/mergefile/mergefile_' + str(subject_code) + '.csv'
    print(beh_path)
    merged_file = pd.read_csv(beh_path)
    merged_file.head()
    #print(merged_file.shape) #(480, 53)
    
    return merged_file,beh_path

merged_file, beh_path= load_beh(server_folder, subject_code)
#print(beh_path)
# 484 rows in the dataframe --> B/NB tones

/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/eeg_beh_results/mergefile/mergefile_sub-062.csv


# NB and B

### CREATE EEG epochs for nb and b

In [14]:
def create_indiv_epochs(epochs, events, event_id, merged_file, epoch_data, condition_label):

    # Convert condition label to event ID and find corresponding epochs
    event_value = event_id[condition_label]
    condition_events = events[:, 2] == event_value

    # Identify which epochs were retained (not dropped due to annotation)
    retained_epochs = epochs.selection
    undropped_condition_events = np.isin(np.where(condition_events)[0], retained_epochs)

    # Filter the behavioral data to match the undropped, condition-specific epochs
    b_epochs_indices = merged_file['TrueBoundary.x'][undropped_condition_events] == 'B'
    nb_epochs_indices = merged_file['TrueBoundary.x'][undropped_condition_events] == 'NB'

    # Select epochs based on behavioral indices
    b_epochs = epoch_data[b_epochs_indices, :, :]
    nb_epochs = epoch_data[nb_epochs_indices, :, :]

    # Print shapes of the epochs for boundary and non-boundary conditions
    #print(f"The shape of nb_epochs is {nb_epochs.shape}")
    #print(f"The shape of b_epochs is {b_epochs.shape}")

    # Averaging all B, NB, NB controlled trials
    b_ERP = np.nanmean(b_epochs, axis=0)
    nb_ERP = np.nanmean(nb_epochs, axis=0)

    # Print ERP shapes
    print(nb_ERP.shape)
    print(b_ERP.shape)

    return b_ERP, nb_ERP, electrode_names, b_epochs, nb_epochs

#call function
b_ERP, nb_ERP, electrode_names, b_epochs, nb_epochs= create_indiv_epochs(epochs, events, event_id, merged_file, epoch_data, condition_label)


/var/folders/np/qpdx19z16vj6zqjjk4pnx35r0000gn/T/ipykernel_4427/1706219027.py:24: RuntimeWarning: Mean of empty slice
  b_ERP = np.nanmean(b_epochs, axis=0)


(63, 1101)
(63, 1101)


/var/folders/np/qpdx19z16vj6zqjjk4pnx35r0000gn/T/ipykernel_4427/1706219027.py:25: RuntimeWarning: Mean of empty slice
  nb_ERP = np.nanmean(nb_epochs, axis=0)


In [17]:
# Setup for plotting for individual ERPs

fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(20, 10)) 

# Flatten the axs array for easy iteration
axs = axs.flatten()

for ax, electrode in zip(axs, electrode_names):
    electrode_idx = np.array(data_channels) == electrode

    ax.plot(b_ERP[electrode_idx,:].T) # b_ERP[electrode_idx,:].squeeze()
    ax.plot(nb_ERP[electrode_idx,:].T)
    ax.legend(['Boundary','NonBoundary'])
    ax.set_xlabel('Time Rel. Stimulus Onset (ms)'); 
    ax.set_ylabel('Voltage $\mu$V');
    ax.set_xticks([0,100,200,400,600,800,1000]); #time point of the vector
    ax.set_xticklabels([-100,0,100,300,500,700,900]); #actural time point
    ax.set_title(f'{condition_label}_{electrode}')
    ax.set_ylim(-10, 13)

# Remove unused subplots when cols are not filled
for i in range(len(electrode_names), len(axs)):
    fig.delaxes(axs[i])

# Adjust the space between the subplots
plt.subplots_adjust(wspace=0.3, hspace=0.3)

figure_fname = os.path.join(data_folder, subject_code, 'eeg','analysis','Boundary vs NonBoundary'
                            f'{subject_code}_{condition_label}_All_Electrodes.pdf')
# save the file 
plt.savefig(figure_fname, bbox_inches='tight')

In [20]:
import scipy.stats as spst

subject_list = ['sub-036', 'sub-037','sub-039', 'sub-040', 'sub-041', 'sub-042', 'sub-043',
                'sub-044', 'sub-045','sub-046', 'sub-047', 'sub-048', 'sub-049',
                'sub-050','sub-051', 'sub-052','sub-053', 'sub-054','sub-055','sub-056','sub-057','sub-058','sub-061']

nsubjects = len(subject_list)
print(nsubjects)
nelectrodes=63
samples=1101

23


In [13]:
#subject_list = [f'sub-{i:03}' for i in range(51, 55)]
#print(subject_list)

['sub-051', 'sub-052', 'sub-053', 'sub-054']


## Comparison of P300 Amplitude Between Boundary and Non-Boundary trials

In [29]:
group_boundary = np.zeros((nsubjects, nelectrodes, samples))
group_nonboundary = np.zeros((nsubjects, nelectrodes, samples))

all_b_ERP = []
all_nb_ERP = []

for ctr_subj,isubj in enumerate(subject_list):
    
    # do all the stuff above for a single subject:
    raw, data_channels = load_and_process_eeg_data(data_folder, isubj)
    events, event_counts, event_id = read_event_markers(raw)
    epoch_data, condition_label, epochs = create_epochs(raw,events, event_id)
    merged_file, beh_path= load_beh(server_folder, isubj)
    b_ERP, nb_ERP, electrode_names = create_indiv_epochs(epochs, events, event_id, merged_file, epoch_data, condition_label)
    
    # shape (nsubjects x nelectrodes x samples)
    group_boundary[ctr_subj,:,:] = b_ERP
    group_nonboundary[ctr_subj,:,:] = nb_ERP
    # group_nonboundary => shape (nsubjects x nelectrodes x time in milliseconds)

#save the results to local directory

group_boundary_path = '/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/eeg_postprocessing/group_ERPs/group_boundary.npy'
group_nonboundary_path = '/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/eeg_postprocessing/group_ERPs/group_nonboundary.npy'

# Save the arrays
np.save(group_boundary_path, group_boundary)
np.save(group_nonboundary_path, group_nonboundary)

Opening raw data file /Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/sub-036/eeg/postproc/sub-036_filtered_annot_crop.fif...


/var/folders/np/qpdx19z16vj6zqjjk4pnx35r0000gn/T/ipykernel_59585/93895693.py:7: RuntimeWarning: This filename (/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/sub-036/eeg/postproc/sub-036_filtered_annot_crop.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(fname)


    Range : 170000 ... 5330000 =    170.000 ...  5330.000 secs
Ready.
Used Annotations descriptions: ['Bit0/B  1', 'Bit1/B  1', 'Bit2/B  1', 'Bit3/B  1', 'Bit4/B  1', 'Bit5/B  1', 'Bit6/B  1']

Event counts: [480 480  15 105 105 270   1]
Not setting metadata
480 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 480 events and 1101 original time points ...
0 bad epochs dropped
/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/eeg_beh_results/mergefile/mergefile_sub-036.csv
(63, 1101)
(63, 1101)
Opening raw data file /Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/sub-037/eeg/postproc/sub-037_filtered_annot_crop.fif...


/var/folders/np/qpdx19z16vj6zqjjk4pnx35r0000gn/T/ipykernel_59585/93895693.py:7: RuntimeWarning: This filename (/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/sub-037/eeg/postproc/sub-037_filtered_annot_crop.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(fname)


    Range : 0 ... 4900000 =      0.000 ...  4900.000 secs
Ready.
Used Annotations descriptions: ['Bit0/B  1', 'Bit1/B  1', 'Bit2/B  1', 'Bit3/B  1', 'Bit4/B  1', 'Bit5/B  1', 'Bit6/B  1', 'New Segment/']

Event counts: [480 480  15 105 105 270   1   1]
Not setting metadata
480 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 480 events and 1101 original time points ...
0 bad epochs dropped
/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/eeg_beh_results/mergefile/mergefile_sub-037.csv
(63, 1101)
(63, 1101)
Opening raw data file /Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/sub-039/eeg/postproc/sub-039_filtered_annot_crop.fif...


...

    Range : 0 ... 5424599 =      0.000 ...  5424.599 secs
Ready.
Used Annotations descriptions: ['Bit0/B  1', 'Bit1/B  1', 'Bit2/B  1', 'Bit3/B  1', 'Bit4/B  1', 'Bit5/B  1', 'Bit6/B  1', 'New Segment/']

Event counts: [480 480  15 105 105 270   1   1]
Not setting metadata
480 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 480 events and 1101 original time points ...
0 bad epochs dropped
/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/eeg_beh_results/mergefile/mergefile_sub-058.csv
(63, 1101)
(63, 1101)
Opening raw data file /Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/sub-061/eeg/postproc/sub-061_filtered_annot_crop.fif...


/var/folders/np/qpdx19z16vj6zqjjk4pnx35r0000gn/T/ipykernel_59585/93895693.py:7: RuntimeWarning: This filename (/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/sub-061/eeg/postproc/sub-061_filtered_annot_crop.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(fname)


    Range : 0 ... 5740139 =      0.000 ...  5740.139 secs
Ready.
Used Annotations descriptions: ['Bit0/B  1', 'Bit1/B  1', 'Bit2/B  1', 'Bit3/B  1', 'Bit4/B  1', 'Bit5/B  1', 'Bit6/B  1', 'New Segment/']

Event counts: [480 480  15 105 105 270   2   1]
Not setting metadata
480 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 480 events and 1101 original time points ...
0 bad epochs dropped
/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/eeg_beh_results/mergefile/mergefile_sub-061.csv


/var/folders/np/qpdx19z16vj6zqjjk4pnx35r0000gn/T/ipykernel_59585/3378991457.py:24: RuntimeWarning: Mean of empty slice
  b_ERP = np.nanmean(b_epochs, axis=0)
/var/folders/np/qpdx19z16vj6zqjjk4pnx35r0000gn/T/ipykernel_59585/3378991457.py:25: RuntimeWarning: Mean of empty slice
  nb_ERP = np.nanmean(nb_epochs, axis=0)


(63, 1101)
(63, 1101)


In [4]:
######
# load the saved file from local directory
group_boundary_path = '/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/eeg_postprocessing/group_ERPs/group_boundary.npy'
group_nonboundary_path = '/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/eeg_postprocessing/group_ERPs/group_nonboundary.npy'

group_boundary = np.load(group_boundary_path)
group_nonboundary = np.load(group_nonboundary_path)

#check the data shape (nsubjects x nelectrodes x time in milliseconds)
print(group_boundary.shape)
print(group_nonboundary.shape)


(23, 63, 1101)
(23, 63, 1101)


In [5]:
print(data_channels)

electrode_of_interest= 'Pz' 
electrode_index = data_channels.index(electrode_of_interest)  # Find the index of 'Cz'
print(electrode_index)

['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'C1', 'C5', 'TP7', 'CP3', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2', 'FC4', 'FT8', 'F6', 'AF8', 'AF4', 'F2']
12


### Paired T-tests

In [6]:
# calculate the mean amplitudes across time window of interests
 ## selected time window: 300-700 ms after stimuli onsets
start_window = 400 
end_window = 800

(tstat,pval) = spst.ttest_rel(np.nanmean(group_boundary[:,:,start_window:end_window],axis=-1),
                              np.nanmean(group_nonboundary[:,:,start_window:end_window],axis=-1),
                              axis=0, nan_policy='omit')

#calculate mean_b and mean_nb: P3 amplitude for each participants; shape: participants_num * electrode

mean_b= np.nanmean(group_boundary[:,:,start_window:end_window],axis=2) # shape: participants_num * electrode
print('mean_b')
print(mean_b[:, electrode_index])
mean_nb= np.nanmean(group_nonboundary[:,:,start_window:end_window],axis=2)
print('mean_nb')
print(mean_nb[:, electrode_index])

# Initialize an empty list to store electrode names with pval < 0.05
significant_electrodes = []

# Print the results
for i, name in enumerate(data_channels):
    # Check if the p-value is less than 0.05
    if pval[i] < 0.05:
        print(f"Electrode: {name}, T-stat: {tstat[i]:.4f}, P-val: {pval[i]:.4f} *")
        significant_electrodes.append(True)
    else:
        print(f"Electrode: {name}, T-stat: {tstat[i]:.4f}, P-val: {pval[i]:.4f}")
        significant_electrodes.append(False)

#significant_electrodes


mean_b
[11.10282016  2.31258893  1.3420788   3.44476069  3.17003463  4.35148043
  6.89619537 11.66012866  8.44819013  9.06103652  5.39123536  9.53548559
  7.94080061  2.81544983  8.23220189 10.83435963  3.07786072 10.61950859
  2.69673068  4.49284669  9.69222574  6.24343169 10.25060557]
mean_nb
[ 2.18740572 -0.05346134 -0.3219777   0.84423868  3.25596247  2.21196895
 -1.21587063  1.76914016  1.8185009   2.5540849   0.34253871  1.15539624
  0.66601327  1.03772528  1.60358851  3.47155518 -0.15268791  0.44713894
  0.70907093  1.31962593  2.70154585 -0.36147899  2.07273001]
Electrode: Fp1, T-stat: -0.9701, P-val: 0.3425
Electrode: Fz, T-stat: 2.3024, P-val: 0.0312 *
Electrode: F3, T-stat: -0.8354, P-val: 0.4129
Electrode: F7, T-stat: -0.6183, P-val: 0.5428
Electrode: FT9, T-stat: -0.0929, P-val: 0.9268
Electrode: FC5, T-stat: 0.5109, P-val: 0.6145
Electrode: FC1, T-stat: 3.7133, P-val: 0.0012 *
Electrode: C3, T-stat: 3.5322, P-val: 0.0019 *
Electrode: T7, T-stat: 0.5005, P-val: 0.6217
Elec

/var/folders/np/qpdx19z16vj6zqjjk4pnx35r0000gn/T/ipykernel_88356/579013444.py:6: RuntimeWarning: Mean of empty slice
  (tstat,pval) = spst.ttest_rel(np.nanmean(group_boundary[:,:,start_window:end_window],axis=-1),
/var/folders/np/qpdx19z16vj6zqjjk4pnx35r0000gn/T/ipykernel_88356/579013444.py:7: RuntimeWarning: Mean of empty slice
  np.nanmean(group_nonboundary[:,:,start_window:end_window],axis=-1),
/var/folders/np/qpdx19z16vj6zqjjk4pnx35r0000gn/T/ipykernel_88356/579013444.py:12: RuntimeWarning: Mean of empty slice
  mean_b= np.nanmean(group_boundary[:,:,start_window:end_window],axis=2) # shape: participants_num * electrode
/var/folders/np/qpdx19z16vj6zqjjk4pnx35r0000gn/T/ipykernel_88356/579013444.py:15: RuntimeWarning: Mean of empty slice
  mean_nb= np.nanmean(group_nonboundary[:,:,start_window:end_window],axis=2)


### Graph the group ERPs

In [33]:
# calculate grand_ERPs (ERP average across all participants)
grand_ERP_nb = np.nanmean (group_nonboundary, axis=0) #(electrodes) * time points (63, 1101)
grand_ERP_b = np.nanmean (group_boundary, axis=0)     

# Save the arrays

# Define the file paths
grand_boundary_path = '/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/eeg_postprocessing/group_ERPs/grand_boundary.csv'
grand_nonboundary_path = '/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/eeg_postprocessing/group_ERPs/grand_nonboundary.csv'

# Save the arrays to CSV
np.savetxt(grand_boundary_path, grand_ERP_b, delimiter=',')
np.savetxt(grand_nonboundary_path, grand_ERP_nb, delimiter=',')


In [91]:
#Count the number of participants with NaN

# electrodes have NaN values
#nan_electrodes = np.isnan(group_nonboundary).any(axis=(0, 2))

# Get the indices of electrodes with NaN values
#electrodes_with_nan = np.where(nan_electrodes)[0]

# Initialize a dictionary to store the count of participants with NaN values for each electrode
#nan_count_per_electrode = {}

# Iterate over each electrode that has NaN values
#for electrode in electrodes_with_nan:
#    nan_count = np.isnan(group_nonboundary[:, electrode, :]).any(axis=1).sum()
#    nan_count_per_electrode[electrode] = nan_count

    #for electrode, count in nan_count_per_electrode.items():
#    print(f"Electrode {electrode} has {count} participants with NaN values.")



In [7]:

# Assuming group_nonboundary has the shape (participants, electrodes, time points)
number_of_electrodes = group_nonboundary.shape[1]

# Initialize a 2D array to store the electrode index and the count of non-NaN participants
count_per_electrode = np.zeros((number_of_electrodes, 2), dtype=int)

# Iterate over all electrodes
for electrode in range(number_of_electrodes):
    # Count the number of non-NaN participants for this electrode
    count = (~np.isnan(group_nonboundary[:, electrode, :])).all(axis=1).sum()
    count_per_electrode[electrode] = [electrode, count]

# Now non_nan_count_per_electrode is a 2D array with the first column as electrode index
# and the second column as the count of non-NaN participants for that electrode
#print("Count of each electrode")
#print(count_per_electrode)


In [8]:
import math

# shape should be (number_of_electrodes, number_of_time_points)
std_nb = np.nanstd(group_nonboundary, axis=0, ddof=1)  
std_b = np.nanstd(group_boundary, axis=0, ddof=1)  
# Assuming std is correctly calculated before this snippet
grand_ERP_nb_errorbar = np.zeros_like(std_nb) 
grand_ERP_b_errorbar = np.zeros_like(std_b) 

## nonboundary_errorbar
for electrode in range(len(count_per_electrode)):
    count = count_per_electrode[electrode, 1]  # Access the count for the electrode
    grand_ERP_nb_errorbar[electrode, :] = std_nb[electrode, :] / math.sqrt(count)
    #print(f"In nb condition, electrode {electrode} has {count} valid (non-NaN) observations.")


## boundary_errorbar
for electrode in range(len(count_per_electrode)):
    count = count_per_electrode[electrode, 1]  # Access the count for the electrode
    grand_ERP_b_errorbar[electrode, :] = std_b[electrode, :] / math.sqrt(count)
    #print(f"In b boundition, Electrode {electrode} has {count} valid (non-NaN) observations.")


In [16]:
import seaborn as sns
import matplotlib.patches as mpatches

# Set Seaborn style
sns.set_style("whitegrid")

time_points = np.arange(grand_ERP_nb.shape[1])  # Assuming time points are linearly spaced

## grand_ERPs without error bars
fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(20, 10)) 

# Flatten the axs array for easy iteration
axs = axs.flatten()

for ax, electrode in zip(axs, electrode_names):
    electrode_idx = np.array(data_channels) == electrode
    # Extract the mean ERP and SEM for the current electrode
    mean_erp_b = grand_ERP_b[electrode_idx, :].flatten()
    sem_b = grand_ERP_b_errorbar[electrode_idx, :].flatten()
    
    mean_erp_nb = grand_ERP_nb[electrode_idx, :].flatten()
    sem_nb = grand_ERP_nb_errorbar[electrode_idx, :].flatten()
    
    # Plot mean ERP for boundary condition
    ax.plot(time_points, mean_erp_b, label='Boundary')
    # Add shaded SEM area for boundary condition
    ax.fill_between(time_points, mean_erp_b - sem_b, mean_erp_b + sem_b, alpha=0.2)
    
    # Plot mean ERP for nonboundary condition
    ax.plot(time_points, mean_erp_nb, label='NonBoundary')
    # Add shaded SEM area for nonboundary condition
    ax.fill_between(time_points, mean_erp_nb - sem_nb, mean_erp_nb + sem_nb, alpha=0.2)
    
    # Set the rest of the plot attributes
    ax.legend()
    ax.set_xlabel('Time Rel. Stimulus Onset (ms)')
    ax.set_ylabel('Voltage $\mu$V')
    ax.set_xticks([0, 100, 200, 400, 600, 800, 1000])  # Ensure these match your data
    ax.set_xticklabels([-100, 0, 100, 300, 500, 700, 900])  # Ensure these match your data
    ax.set_title(f'{condition_label}_{electrode}')
    ax.set_ylim(-10, 20)
    legend = ax.get_legend()
    if legend:
        legend.remove()
        
    # After plotting is done for all electrodes, create a legend for the error bars only
    boundary_patch = mpatches.Patch(color='blue', label='Boundary Error')
    nonboundary_patch = mpatches.Patch(color='orange',label='NonBoundary Error')
    # Add the custom legend to the current subplot
    ax.legend(handles=[boundary_patch, nonboundary_patch])
    
# Save and show the figure
figure_fname = os.path.join(data_folder, 'eeg_postprocessing', 'group_ERPs', 'Grand_ERPs_All_Electrodes_with_Error_Bars.pdf')
plt.savefig(figure_fname, bbox_inches='tight')


NameError: name 'grand_ERP_nb' is not defined

In [96]:
## grand_ERPs without error bars
fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(20, 10)) 

# Flatten the axs array for easy iteration
axs = axs.flatten()

for ax, electrode in zip(axs, electrode_names):
    electrode_idx = np.array(data_channels) == electrode
    ax.plot(grand_ERP_b[electrode_idx,:].T)
    ax.plot(grand_ERP_nb[electrode_idx,:].T) # nb_ERP[electrode_idx,:].squeeze()
    ax.legend(['Boundary','NonBoundary'])
    ax.set_xlabel('Time Rel. Stimulus Onset (ms)'); 
    ax.set_ylabel('Voltage $\mu$V');
    ax.set_xticks([0,100,200,400,600,800,1000]); #time point of the vector
    ax.set_xticklabels([-100,0,100,300,500,700,900]); #actural time point
    ax.set_title(f'{condition_label}_{electrode}')
    ax.set_ylim(-10, 20)

# Remove unused subplots when cols are not filled
for i in range(len(electrode_names), len(axs)):
    fig.delaxes(axs[i])

# Adjust the space between the subplots
plt.subplots_adjust(wspace=0.3, hspace=0.3)

figure_fname = os.path.join(data_folder, 'eeg_postprocessing','group_ERPs' ,
                            f'Grand_ERPs_All_Electrodes.pdf')
# save the file 
plt.savefig(figure_fname, bbox_inches='tight')

## Topomap with sig electrodes

In [37]:

# calculate grand_mean (P300 amplitude average across all participants)
grand_mean_diff = np.mean(grand_ERP_b [:,start_window:end_window],axis=1)- np.mean(grand_ERP_nb [:,start_window:end_window],axis=1)

# exclude broken electrode
grand_mean_diff[25]=0 #broken electrodes
grand_mean_diff[2]=0

#adjust the unit
grand_mean_diff=grand_mean_diff/1e6

#print out the difference
grand_mean_diff

array([-2.52196601e-06,  2.47138293e-06,  0.00000000e+00, -1.03897948e-06,
       -1.19383406e-07,  5.78049309e-07,  3.47456313e-06,  3.06854675e-06,
        4.50340390e-07,  3.91212999e-07,  2.29131286e-06,  5.07344093e-06,
        5.45866534e-06,  4.02661478e-06,  2.35775063e-06,  1.82247216e-06,
        1.29395948e-06,  1.71112304e-06,  4.12001407e-06,  1.44766407e-06,
       -2.40896940e-07,  2.26050886e-06,  5.06847404e-06,  5.17938227e-06,
        2.85334366e-06,  0.00000000e+00, -2.86551338e-06, -2.18938257e-07,
        3.26475299e-06,  3.55672516e-07, -2.78899537e-06, -3.46071799e-06,
       -1.79626660e-06, -3.48841388e-07,  1.89510485e-07,  1.96522685e-06,
       -2.63000736e-07,  2.66914384e-08,  1.94575187e-06,  4.51418531e-06,
        1.31454021e-06,  1.26348894e-06,  3.83959441e-06,  4.93997577e-06,
        3.58571992e-06,  2.10013512e-06,  3.10950377e-06,  3.55496064e-06,
        3.11900007e-06,  1.75700199e-06,  3.02386831e-06,  4.85089386e-06,
        5.73286274e-06,  

In [39]:
info = raw.info

# Create the EvokedArray, ensuring grand_mean_b is 2D (n_channels, 1)
grand_mean_2d = grand_mean_diff[:, np.newaxis]  # Add an axis to make it 2D

# Create the Evoked object
evoked_grand_mean = mne.EvokedArray(grand_mean_2d, info, tmin=0)

# Define a threshold and create the mask
#mask = evoked_grand_mean.data > 1e-13
significant_electrodes=np.array(significant_electrodes)
mask=significant_electrodes.reshape((63,1))

mask_params = dict(markersize=6, markerfacecolor="y")

# Plot the topomap for the grand mean across participants


im = evoked_grand_mean.plot_topomap(times=[0], ch_type='eeg', time_unit='s', colorbar=True,
                               mask=mask, mask_params=mask_params)

im.axes[0].set_title('300-700ms')
#plt.show()
figure_fname = os.path.join(data_folder, 'eeg_postprocessing', 'topo_maps',
                            f'Topo_maps_sig_electrodes.pdf')

# save the file 
plt.savefig(figure_fname, bbox_inches='tight')

# Close the figure window
plt.close()  


## Does the group_boundary - group_nonboundary negatively correlate with the b-nb temporal order? but maybe not at all the electrodes? Pz?

In [111]:
# Calculate the difference in mean amplitudes between conditions
difference_b_nb = mean_b - mean_nb # with a shape (subject, electrodes)
difference_b_nb.shape

print(difference_b_nb[:,18])

[ 6.61349773  2.23201406  2.22010389  3.00574922 -0.67118476  1.39804191
  7.18464127  4.03649968  3.33858257  5.7039335   1.59688445  7.13159919
  4.6605652   0.21337824  6.3314789   6.77373228  2.67736654  8.39229089
  2.00875845  2.32939313  6.66286301  3.57782073  7.34231353]


In [112]:
# load beh_temporal_order file 
beh_order_accuracy = pd.read_csv('/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/eeg_beh_results/group_results/order_accuracy.csv')
#print(beh_order_accuracy.shape) 

#calculate beh B-NB
beh_order_accuracy['B-NB'] = (beh_order_accuracy['B'] - beh_order_accuracy['NB'])
#beh_order_accuracy['B-NB'] = ((beh_order_accuracy['B'] - beh_order_accuracy['NB']))/beh_order_accuracy['NB']

#remove any subject in the beh data but not in the eeg subject list

beh_order_accuracy['Subject']= ["sub-0" + s.split()[1] for s in beh_order_accuracy['Subject']]
beh_order_accuracy = beh_order_accuracy[beh_order_accuracy['Subject'].isin(subject_list)]

#check if the leng of the lists matched
print("those two numbers should be the same")
print(len(subject_list))
print(len(beh_order_accuracy['Subject'])) # those two numbers should be the same
beh_order_accuracy.head(30)

those two numbers should be the same
23
23


,Metrics,B,Correct.order,Correct.Source,Incorrect.order,Incorrect.Source,NB,Subject,B-NB
0,TemporalOrderAccuracy,0.571429,NaN,NaN,NaN,NaN,0.678571,sub-036,-0.107143
1,TemporalOrderAccuracy,0.357143,NaN,NaN,NaN,NaN,0.500000,sub-037,-0.142857
3,TemporalOrderAccuracy,0.571429,NaN,NaN,NaN,NaN,0.750000,sub-039,-0.178571
4,TemporalOrderAccuracy,0.523810,NaN,NaN,NaN,NaN,0.714286,sub-040,-0.190476
5,TemporalOrderAccuracy,0.404762,NaN,NaN,NaN,NaN,0.642857,sub-041,-0.238095
6,TemporalOrderAccuracy,0.738095,NaN,NaN,NaN,NaN,0.785714,sub-042,-0.047619
7,TemporalOrderAccuracy,0.690476,NaN,NaN,NaN,NaN,0.607143,sub-043,0.083333
8,TemporalOrderAccuracy,0.666667,NaN,NaN,NaN,NaN,0.750000,sub-044,-0.083333
9,TemporalOrderAccuracy,0.500000,NaN,NaN,NaN,NaN,0.696429,sub-045,-0.196429
10,TemporalOrderAccuracy,0.595238,NaN,NaN,NaN,NaN,0.875000,sub-046,-0.279762


In [113]:
correlation_results = {}
from scipy.stats import pearsonr

# Iterate over each electrode
for i, electrode in enumerate(data_channels):
    # Extract the difference scores for this electrode across subjects
    diff_scores = difference_b_nb[:, i]
    
    # filter out the NaNs
    valid_indices = ~np.isnan(diff_scores) & ~np.isnan(beh_order_accuracy['B-NB'])
    diff_scores_filtered = diff_scores[valid_indices]
    beh_order_accuracy_filtered = beh_order_accuracy['B-NB'][valid_indices]
    
    #Run the correlation
    if len(diff_scores_filtered) > 1:  # Ensure there are at least two data points
        correlation, p_value = pearsonr(diff_scores_filtered, beh_order_accuracy_filtered)
        
        # Store the results
        correlation_results[electrode] = (correlation, p_value)
    else:
        correlation_results[electrode] = (np.nan, np.nan)  # Handle case with insufficient data

# print the correlation results
for electrode, (correlation, p_value) in correlation_results.items():
    print(f"Electrode: {electrode}, Correlation: {correlation:.4f}, P-value: {p_value:.4f}")


Electrode: Fp1, Correlation: 0.2947, P-value: 0.1723
Electrode: Fz, Correlation: 0.2633, P-value: 0.2247
Electrode: F3, Correlation: 0.3401, P-value: 0.1214
Electrode: F7, Correlation: 0.2610, P-value: 0.2291
Electrode: FT9, Correlation: 0.3397, P-value: 0.1128
Electrode: FC5, Correlation: 0.2271, P-value: 0.2973
Electrode: FC1, Correlation: 0.1787, P-value: 0.4147
Electrode: C3, Correlation: 0.1027, P-value: 0.6410
Electrode: T7, Correlation: 0.2395, P-value: 0.2710
Electrode: TP9, Correlation: 0.2767, P-value: 0.2011
Electrode: CP5, Correlation: 0.0793, P-value: 0.7191
Electrode: CP1, Correlation: 0.0862, P-value: 0.6959
Electrode: Pz, Correlation: 0.0874, P-value: 0.6917
Electrode: P3, Correlation: 0.0548, P-value: 0.8037
Electrode: P7, Correlation: 0.1477, P-value: 0.5013
Electrode: O1, Correlation: 0.3307, P-value: 0.1232
Electrode: Oz, Correlation: 0.3454, P-value: 0.1065
Electrode: O2, Correlation: 0.2945, P-value: 0.1726
Electrode: P4, Correlation: 0.1292, P-value: 0.5567
Elect

In [114]:
import matplotlib.pyplot as plt

electrode_of_interest= ['Cz','Fz','Pz','P3','P4']

# Setup for plotting exactly 6 plots
num_plots = len(electrode_of_interest)
num_cols = 3  # Set the number of columns to 3 for 6 plots to be arranged in 2 rows
num_rows = 2  # Two rows to accommodate 6 plots

# Create a large figure to hold all subplots
plt.figure(figsize=(num_cols * 5, num_rows * 5))

# Iterate over each electrode of interest to create a subplot for each
for i, electrode in enumerate(electrode_of_interest):
    ax = plt.subplot(num_rows, num_cols, i + 1)
    
    # Find the index of the current electrode in the data_channels list
    electrode_index = data_channels.index(electrode)
    
    # Extract the difference scores for this electrode across subjects
    diff_scores = difference_b_nb[:, electrode_index]
    
    # Scatter plot of difference scores vs. B-NB accuracy scores
    ax.scatter(diff_scores, beh_order_accuracy['B-NB'], alpha=0.7)
    
    # Annotate the plot with the correlation coefficient and p-value
    # Ensure that correlation and p_value are correctly retrieved for each electrode
    correlation, p_value = correlation_results[electrode]
    ax.text(0.05, 0.95, f'r={correlation:.4f}, p={p_value:.4f}', transform=ax.transAxes, 
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # Set plot titles and labels
    ax.set_title(f"Electrode: {electrode}")
    ax.set_xlabel('Difference B-NB (μV)')
    ax.set_ylabel('B-NB Score (%)')

# Ensure a clean layout
plt.tight_layout()


figure_fname = os.path.join(data_folder, 'eeg_postprocessing','correlation','order_correlation.pdf')
# save the file 
plt.savefig(figure_fname)

In [88]:
diff_scores = difference_b_nb[:, 23]
print(diff_scores)

[ 6.83659794e+00  5.91854040e-01  1.33300950e+00  4.58863803e-03
  2.81292226e+00  9.79932275e+00  8.16856910e+00  4.18177497e+00
  7.46696318e+00  5.17843412e+00  1.07683208e+01  4.94957856e+00
  2.01513482e+00 -1.54615909e+02  1.04333133e+01  3.09005067e+00
  1.25094255e+01  8.56321628e-01]


## Does the group_boundary - group_nonboundary negatively correlate with the b-nb source memory? but maybe not at all the electrodes? Pz?

In [117]:
# load beh_temporal_order file 
beh_source_accuracy = pd.read_csv('/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/eeg_beh_results/group_results/source_accuracy.csv')
#print(beh_order_accuracy.shape) 

#calculate beh B-NB
beh_source_accuracy['B-NB'] = (beh_source_accuracy['B'] - beh_source_accuracy['NB'])
#
#beh_order_accuracy['B-NB'] = ((beh_order_accuracy['B'] - beh_order_accuracy['NB']))/beh_order_accuracy['NB']

#remove any subject in the beh data but not in the eeg subject list

beh_source_accuracy['Subject']= ["sub-0" + s.split()[1] for s in beh_source_accuracy['Subject']]

#remove any subject in the beh data but not in the eeg subject list
beh_source_accuracy = beh_source_accuracy[beh_source_accuracy['Subject'].isin(subject_list)]
print(len(subject_list))
print(len(beh_source_accuracy['Subject'])) # those two numbers should be the same
beh_source_accuracy.head(5)


23
23


,Metrics,B,Correct.order,Correct.Source,FT,Incorrect.order,Incorrect.Source,NB,PreB,Subject,B-NB
0,SourceMemoryAccuracy,0.952381,NaN,NaN,0.785714,NaN,NaN,0.807143,0.750000,sub-036,0.145238
1,SourceMemoryAccuracy,0.738095,NaN,NaN,0.642857,NaN,NaN,0.664286,0.660714,sub-037,0.073809
3,SourceMemoryAccuracy,0.714286,NaN,NaN,0.571429,NaN,NaN,0.678571,0.607143,sub-039,0.035714
4,SourceMemoryAccuracy,0.833333,NaN,NaN,0.857143,NaN,NaN,0.671429,0.678571,sub-040,0.161905
5,SourceMemoryAccuracy,0.666667,NaN,NaN,0.642857,NaN,NaN,0.657143,0.571429,sub-041,0.009524


In [119]:
correlation_results = {}
from scipy.stats import pearsonr

# Iterate over each electrode
for i, electrode in enumerate(data_channels):
    # Extract the difference scores for this electrode across subjects
    diff_amplitude = difference_b_nb[:, i]
    
    # filter out the NaNs
    valid_indices = ~np.isnan(diff_scores) & ~np.isnan(beh_source_accuracy['B-NB'])
    diff_scores_filtered = diff_scores[valid_indices]
    beh_source_accuracy_filtered = beh_source_accuracy['B-NB'][valid_indices]
    
    #Run the correlation
    if len(diff_scores_filtered) > 1:  # Ensure there are at least two data points
        correlation, p_value = pearsonr(diff_scores_filtered, beh_source_accuracy_filtered)
        
        # Store the results
        correlation_results[electrode] = (correlation, p_value)
    else:
        correlation_results[electrode] = (np.nan, np.nan)  # Handle case with insufficient data


# print the correlation results
for electrode, (correlation, p_value) in correlation_results.items():
    print(f"Electrode: {electrode}, Correlation: {correlation:.4f}, P-value: {p_value:.4f}")
    
---
correlation_results = {}
from scipy.stats import pearsonr

# Iterate over each electrode
for i, electrode in enumerate(data_channels):
    # Extract the difference scores for this electrode across subjects
    diff_scores = difference_b_nb[:, i]
    
    # filter out the NaNs
    valid_indices = ~np.isnan(diff_scores) & ~np.isnan(beh_order_accuracy['B-NB'])
    diff_scores_filtered = diff_scores[valid_indices]
    beh_order_accuracy_filtered = beh_order_accuracy['B-NB'][valid_indices]
    
    #Run the correlation
    if len(diff_scores_filtered) > 1:  # Ensure there are at least two data points
        correlation, p_value = pearsonr(diff_scores_filtered, beh_order_accuracy_filtered)
        
        # Store the results
        correlation_results[electrode] = (correlation, p_value)
    else:
        correlation_results[electrode] = (np.nan, np.nan)  # Handle case with insufficient data

# print the correlation results
for electrode, (correlation, p_value) in correlation_results.items():
    print(f"Electrode: {electrode}, Correlation: {correlation:.4f}, P-value: {p_value:.4f}")

Electrode: Fp1, Correlation: -0.0881, P-value: 0.6892
Electrode: Fz, Correlation: -0.0881, P-value: 0.6892
Electrode: F3, Correlation: -0.0881, P-value: 0.6892
Electrode: F7, Correlation: -0.0881, P-value: 0.6892
Electrode: FT9, Correlation: -0.0881, P-value: 0.6892
Electrode: FC5, Correlation: -0.0881, P-value: 0.6892
Electrode: FC1, Correlation: -0.0881, P-value: 0.6892
Electrode: C3, Correlation: -0.0881, P-value: 0.6892
Electrode: T7, Correlation: -0.0881, P-value: 0.6892
Electrode: TP9, Correlation: -0.0881, P-value: 0.6892
Electrode: CP5, Correlation: -0.0881, P-value: 0.6892
Electrode: CP1, Correlation: -0.0881, P-value: 0.6892
Electrode: Pz, Correlation: -0.0881, P-value: 0.6892
Electrode: P3, Correlation: -0.0881, P-value: 0.6892
Electrode: P7, Correlation: -0.0881, P-value: 0.6892
Electrode: O1, Correlation: -0.0881, P-value: 0.6892
Electrode: Oz, Correlation: -0.0881, P-value: 0.6892
Electrode: O2, Correlation: -0.0881, P-value: 0.6892
Electrode: P4, Correlation: -0.0881, P-

In [120]:
electrode_of_interest= ['Cz','Fz','Pz','P3','P4']

# Setup for plotting exactly 6 plots
num_plots = len(electrode_of_interest)
num_cols = 3  # Set the number of columns to 3 for 6 plots to be arranged in 2 rows
num_rows = 2  # Two rows to accommodate 6 plots

# Create a large figure to hold all subplots
plt.figure(figsize=(num_cols * 5, num_rows * 5))

# Iterate over each electrode of interest to create a subplot for each
for i, electrode in enumerate(electrode_of_interest):
    ax = plt.subplot(num_rows, num_cols, i + 1)
    
    # Find the index of the current electrode in the data_channels list
    electrode_index = data_channels.index(electrode)
    
    # Extract the difference scores for this electrode across subjects
    diff_scores = difference_b_nb[:, electrode_index]
    
    # Scatter plot of difference scores vs. B-NB accuracy scores
    ax.scatter(diff_scores, beh_source_accuracy['B-NB'], alpha=0.7)
    
    # Annotate the plot with the correlation coefficient and p-value
    # Ensure that correlation and p_value are correctly retrieved for each electrode
    correlation, p_value = correlation_results[electrode]
    ax.text(0.05, 0.95, f'r={correlation:.4f}, p={p_value:.4f}', transform=ax.transAxes, 
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # Set plot titles and labels
    ax.set_title(f"Electrode: {electrode}")
    ax.set_xlabel('Difference B-NB (μV)')
    ax.set_ylabel('B-NB Source accuracy (%)')

# Ensure a clean layout
plt.tight_layout()


figure_fname = os.path.join(data_folder, 'correlation_Source.pdf')
# save the file 
plt.savefig(figure_fname)

## Temporal P300 Amplitude as a Predictor for Boundary Items on Temporal Order Accuracy

### Order accuracy

In [126]:
b_epochs.shape #(trials x electrodes x time points)
nb_epochs.shape #(trials x electrodes x time points)


(360, 63, 1101)

## Temporal order/source memory correct vs incorrect

### Temporal order

### Temporal source memory

#### For all the boundary trials

In [31]:

subject_code= 'sub-052'
def calculate_boundary_erp(subject_code, merged_file, epoch_data):
    
    print("Current subject code: ", subject_code)
    
    #load the current beh merged file:
    merged_file, beh_path=load_beh(server_folder, subject_code)
    
    
    # Identify correct and boundary conditions
    idx_1 = merged_file['correct_source'] == 1
    idx_2 = merged_file['TrueBoundary.x'] == 'B'

    # Uncomment to compare idx_1 and idx_2 for debugging
    # comparison = idx_1.compare(idx_2, align_axis=1, keep_shape=True, keep_equal=True, result_names=('self', 'other'))
    # print(comparison)

    # Calculate correct boundary conditions
    boundary_correct_index = (idx_1 == True) & (idx_2 == True)
    boundary_correct_epochs = epoch_data[boundary_correct_index, :, :]
    boundary_correct_ERP = np.nanmean(boundary_correct_epochs, axis=0)
    print(f"Boundary correct epochs shape: {boundary_correct_epochs.shape}")

    # Calculate incorrect boundary conditions
    boundary_incorrect_index = (idx_1 == False) & (idx_2 == True)
    boundary_incorrect_epochs = epoch_data[boundary_incorrect_index, :, :]
    boundary_incorrect_ERP = np.nanmean(boundary_incorrect_epochs, axis=0)
    print(f"Boundary incorrect epochs shape: {boundary_incorrect_epochs.shape}")

    return boundary_correct_ERP, boundary_incorrect_ERP

# function call
boundary_correct_ERP, boundary_incorrect_ERP = calculate_boundary_erp(subject_code, merged_file, epoch_data)

Current subject code:  sub-052
/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/eeg_beh_results/mergefile/mergefile_sub-052.csv


IndexError: boolean index did not match indexed array along dimension 0; dimension is 480 but corresponding boolean dimension is 475

In [18]:
def plot_erp_comparison(boundary_correct_ERP, boundary_incorrect_ERP, electrode_names, data_channels, condition_label, data_folder, subject_code):
    fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(20, 10))

    # Flatten the axs array for easy iteration
    axs = axs.flatten()

    for ax, electrode in zip(axs, electrode_names):
        electrode_idx = np.array(data_channels) == electrode

        ax.plot(boundary_correct_ERP[electrode_idx, :].T)
        ax.plot(boundary_incorrect_ERP[electrode_idx, :].T)
        ax.legend(['B correct', 'B incorrect'])
        ax.set_xlabel('Time Rel. Stimulus Onset (ms)'); ax.set_ylabel('Voltage $\mu$V');
        ax.set_xticks([0, 100, 200, 400, 600, 800, 1000])  # Adjust these to match your data
        ax.set_xticklabels([-100, 0, 100, 300, 500, 700, 900])  # Adjust these to match your data
        ax.set_title(f'{condition_label}_{electrode}')

    # Remove unused subplots when cols are not filled
    for i in range(len(electrode_names), len(axs)):
        fig.delaxes(axs[i])

    fig.suptitle('Comparison of ERPs for Correct and Incorrect Sources at Boundary Condition', fontsize=20)

    # Construct the file name and path
    figure_fname = os.path.join(data_folder, subject_code, 'eeg', 'analysis', f'{subject_code}_boundary_source_All_Electrodes.pdf')
    
    # Save the file
    plt.savefig(figure_fname)
    plt.close()  # Close the figure to free memory

# function call, just for testing purpose
plot_erp_comparison(boundary_correct_ERP, boundary_incorrect_ERP, electrode_names, data_channels, condition_label, data_folder, subject_code)

In [21]:
# conduct the analysis for all participants:
for i in subject_list:
    calculate_boundary_erp(i, merged_file, epoch_data)
    plot_erp_comparison(boundary_correct_ERP, boundary_incorrect_ERP, electrode_names, data_channels, condition_label, data_folder, i)
    

Current subject code:  sub-036
/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/eeg_beh_results/mergefile/mergefile_sub-036.csv


/var/folders/np/qpdx19z16vj6zqjjk4pnx35r0000gn/T/ipykernel_88356/2862802578.py:21: RuntimeWarning: Mean of empty slice
  boundary_correct_ERP = np.nanmean(boundary_correct_epochs, axis=0)
/var/folders/np/qpdx19z16vj6zqjjk4pnx35r0000gn/T/ipykernel_88356/2862802578.py:27: RuntimeWarning: Mean of empty slice
  boundary_incorrect_ERP = np.nanmean(boundary_incorrect_epochs, axis=0)


Boundary correct epochs shape: (40, 63, 1101)
Boundary incorrect epochs shape: (5, 63, 1101)
Current subject code:  sub-037
/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/eeg_beh_results/mergefile/mergefile_sub-037.csv


/var/folders/np/qpdx19z16vj6zqjjk4pnx35r0000gn/T/ipykernel_88356/2862802578.py:21: RuntimeWarning: Mean of empty slice
  boundary_correct_ERP = np.nanmean(boundary_correct_epochs, axis=0)
/var/folders/np/qpdx19z16vj6zqjjk4pnx35r0000gn/T/ipykernel_88356/2862802578.py:27: RuntimeWarning: Mean of empty slice
  boundary_incorrect_ERP = np.nanmean(boundary_incorrect_epochs, axis=0)


Boundary correct epochs shape: (31, 63, 1101)
Boundary incorrect epochs shape: (14, 63, 1101)
Current subject code:  sub-039
/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/eeg_beh_results/mergefile/mergefile_sub-039.csv
Boundary correct epochs shape: (15, 63, 1101)


...

/var/folders/np/qpdx19z16vj6zqjjk4pnx35r0000gn/T/ipykernel_88356/2862802578.py:21: RuntimeWarning: Mean of empty slice
  boundary_correct_ERP = np.nanmean(boundary_correct_epochs, axis=0)
/var/folders/np/qpdx19z16vj6zqjjk4pnx35r0000gn/T/ipykernel_88356/2862802578.py:27: RuntimeWarning: Mean of empty slice
  boundary_incorrect_ERP = np.nanmean(boundary_incorrect_epochs, axis=0)


Boundary correct epochs shape: (40, 63, 1101)
Boundary incorrect epochs shape: (5, 63, 1101)
Current subject code:  sub-045
/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/eeg_beh_results/mergefile/mergefile_sub-045.csv


/var/folders/np/qpdx19z16vj6zqjjk4pnx35r0000gn/T/ipykernel_88356/2862802578.py:21: RuntimeWarning: Mean of empty slice
  boundary_correct_ERP = np.nanmean(boundary_correct_epochs, axis=0)
/var/folders/np/qpdx19z16vj6zqjjk4pnx35r0000gn/T/ipykernel_88356/2862802578.py:27: RuntimeWarning: Mean of empty slice
  boundary_incorrect_ERP = np.nanmean(boundary_incorrect_epochs, axis=0)


Boundary correct epochs shape: (34, 63, 1101)
Boundary incorrect epochs shape: (11, 63, 1101)
Current subject code:  sub-046
/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/eeg_beh_results/mergefile/mergefile_sub-046.csv


IndexError: boolean index did not match indexed array along dimension 0; dimension is 480 but corresponding boolean dimension is 464

## Analysis summer 2023

In [43]:
#NonBoundary
info = raw.info
time_points = np.arange(0, 1.1, 0.100) 
evoked_nb = mne.EvokedArray(nb_ERP, info)
#Boundary condition
evoked_nb.plot_topomap(time_points, ch_type="eeg")


# Path to save the figure
figure_path = os.path.join(data_folder, subject_code, 'eeg','analysis',
                           f'{subject_code}_NonBoundary_topomap.pdf')

# Save the figure
plt.savefig(figure_path, format='pdf')

In [24]:
#Boundary

time_points = np.arange(0, 1.1, 0.100) 
evoked_b = mne.EvokedArray(b_ERP, info)
# Boundary condition
evoked_b.plot_topomap(time_points, ch_type="eeg")

# Path to save the figure
figure_path = os.path.join(data_folder, subject_code, 'eeg','analysis',
                           f'{subject_code}_Boundary_topomap.pdf')

# Save the figure
plt.savefig(figure_path, format='pdf')

In [25]:
#NonBoundary
times = [.150, .250, .3,.4, .5,.6,.7 ,1.1]
fig = evoked_nb.plot_joint(times=times, title='Combination of butterfly plot and topomaps at NonBoundary condition')

figure_path = os.path.join(data_folder, subject_code, 'eeg','analysis',
                           f'{subject_code}__NonBoundary_butterfly_plot.pdf')

# Save the figure
fig.savefig(figure_path, format='pdf')

No projector specified for this dataset. Please consider the method self.add_proj.


KeyboardInterrupt: 

In [ ]:
#Boundary
times = [.150, .250, .3,.4, .5,.6,.7 ,1.1]
fig = evoked_b.plot_joint(times=times, title='Combination of butterfly plot and topomaps at Boundary condition')

figure_path = os.path.join(data_folder, subject_code, 'eeg','analysis',
                           f'{subject_code}__Boundary_butterfly_plot.pdf')

# Save the figure
fig.savefig(figure_path, format='pdf')


###  NB AND B TRIALS 

In [37]:
#rather than generating individual graphs for each channel, 
#I consolidated the information into a single comprehensive graph containing all channels of interest.

fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(20, 10)) 

# Flatten the axs array for easy iteration
axs = axs.flatten()

for ax, electrode in zip(axs, electrode_names):
    electrode_idx = np.array(data_channels) == electrode

    ax.plot(b_ERP[electrode_idx,:].T) # b_ERP[electrode_idx,:].squeeze()
    ax.plot(nb_ERP[electrode_idx,:].T)
    ax.legend(['Boundary','NonBoundary'])
    ax.set_xlabel('Time Rel. Stimulus Onset (ms)'); 
    ax.set_ylabel('Voltage $\mu$V');
    ax.set_xticks([0,100,200,400,600,800,1000]); #time point of the vector
    ax.set_xticklabels([-100,0,100,300,500,700,900]); #actural time point
    ax.set_title(f'{condition_label}_{electrode}')
    ax.set_ylim(-10, 15)

# Remove unused subplots when cols are not filled
for i in range(len(electrode_names), len(axs)):
    fig.delaxes(axs[i])

# Adjust the space between the subplots
plt.subplots_adjust(wspace=0.3, hspace=0.3)

figure_fname = os.path.join(data_folder, subject_code, 'eeg','analysis','Boundary vs NonBoundary'
                            f'{subject_code}_{condition_label}_All_Electrodes.pdf')
# save the file 
plt.savefig(figure_fname, bbox_inches='tight')


#### Controlled

In [81]:
fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(20, 10)) 

# Flatten the axs array for easy iteration
axs = axs.flatten()

for ax, electrode in zip(axs, electrode_name):
    electrode_idx = np.array(data_channels) == electrode

    ax.plot(b_ERP[electrode_idx,:].T) 
    ax.plot(nb_controlled_ERP[electrode_idx,:].T)
    ax.legend(['Boundary','NonBoundary'])
    ax.set_xlabel('Time Rel. Stimulus Onset (ms)'); ax.set_ylabel('Voltage $\mu$V');
    ax.set_xticks([0,100,200,400,600,800,1000]); #time point of the vector
    ax.set_xticklabels([-100,0,100,300,500,700,900]); #actural time point
    ax.set_ylim([-18, 15]) 
    ax.set_title(f'{condition_label}_{electrode}')

# Remove unused subplots when cols are not filled
for i in range(len(electrode_name), len(axs)):
    fig.delaxes(axs[i])

# Adjust the space between the subplots
plt.subplots_adjust(wspace=0.3, hspace=0.3)

#title
title_text = f"Comparison of ERPs for Boundary vs NonBoundary pairs after controlled for trial size - {subject_code}"
fig.suptitle(title_text, fontsize=20)

figure_fname = os.path.join(data_folder, subject_code, 'eeg', 'analysis', 
                            f'CONTROLLED_Boundary vs NonBoundary_{subject_code}_{condition_label}_All_Electrodes.pdf')

# save the file 
plt.savefig(figure_fname, bbox_inches='tight')

## Source Mem.

### Create EEG epochs for source mem

In [119]:
#converting the column names of a pandas DataFrame merged_file into a list.
column_list = merged_file.columns.values.tolist()
#printing out all the unique values in the 'correct_source' colum
print(merged_file.correct_source.unique()) #the list contain three unique values: na, 1,0

[nan  1.  0.]


In [120]:
idx = merged_file['correct_source']==1
correct_source_epochs = epoch_data[idx,:,:]

idx_1 = merged_file['correct_source']==0
incorrect_source_epochs = epoch_data[idx_1,:,:]

## PRINT SHAPE
print(incorrect_source_epochs.shape)
print(correct_source_epochs.shape)

## averaging all 1 and 0 trials
correct_source_ERP = np.nanmean(correct_source_epochs,axis=0)
incorrect_source_ERP = np.nanmean(incorrect_source_epochs,axis=0)

## PRINT SHAPE
print(correct_source_ERP.shape)
print(incorrect_source_ERP.shape)

(82, 63, 1101)
(170, 63, 1101)
(63, 1101)
(63, 1101)


### Create EEG epochs for source mem

##### Federico's original code
for electrode in electrode_name:
    electrode_idx = np.array(data_channels)==electrode

    plt.plot(correct_source_ERP[electrode_idx,:].T) # b_ERP[electrode_idx,:].squeeze()
    plt.plot(incorrect_source_ERP[electrode_idx,:].T)
    plt.legend(['correct','incorrect'])
    plt.xlabel('Time Rel. Stimulus Onset (ms)'); plt.ylabel('Voltage $\mu$V');
    plt.xticks([100,300,500,700,900],[0,200,400,600,800]); 
    plt.title(f'{condition_label}_{electrode}')

    figure_fname = os.path.join(data_folder, subject_code, 'eeg','analysis',
                               f'{subject_code}_source_accuracy_{electrode}.pdf')
    # save the file 
    plt.savefig(figure_fname)
    plt.clf()

In [121]:
fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(20, 10)) 

# Flatten the axs array for easy iteration
axs = axs.flatten()

for ax, electrode in zip(axs, electrode_name):
    electrode_idx = np.array(data_channels) == electrode

    ax.plot(correct_source_ERP[electrode_idx,:].T) 
    ax.plot(incorrect_source_ERP[electrode_idx,:].T)
    ax.legend(['correct','incorrect'])
    ax.set_xlabel('Time Rel. Stimulus Onset (ms)'); 
    ax.set_ylabel('Voltage $\mu$V');
    ax.set_xticks([100,300,500,700,900]); 
    ax.set_xticklabels([0,200,400,600,800]); 
    #ax.set_ylim([-30, 10]) 
    ax.set_title(f'{condition_label}_{electrode}')

# Remove unused subplots when cols are not filled
for i in range(len(electrode_name), len(axs)):
    fig.delaxes(axs[i])

# Adjust the space between the subplots
plt.subplots_adjust(wspace=0.3, hspace=0.3)

#title
fig.suptitle('Comparison of ERPs for Correct and Incorrect Sources', fontsize=20)

figure_fname = os.path.join(data_folder, subject_code, 'eeg','analysis','Source Memory_Correct vs Incorrect'
                            f'{subject_code}_{condition_label}_All_Electrodes.pdf')
# save the file 
plt.savefig(figure_fname, bbox_inches='tight')

### Create EEG epochs for source mem and BOUNDARY TRUE

In [122]:
idx = merged_file['correct_source']==1 #creates a boolean Series where each value is True 
#print(idx)
idx_2 = merged_file['TrueBoundary.x']=='B'
#print(idx_2)
idx.compare(idx_2, align_axis=1, keep_shape=True, keep_equal=True, result_names=('self', 'other'))

boundary_correct_index = (idx == True) & (idx_2 == True)
boundary_correct_epochs = epoch_data[boundary_correct_index,:,:]
boundary_correct_ERP = np.nanmean(boundary_correct_epochs,axis=0)
print(boundary_correct_epochs.shape)

boundary_incorrect_index = (idx == False) & (idx_2 == True)
boundary_incorrect_epochs = epoch_data[boundary_incorrect_index,:,:]
boundary_incorrect_ERP = np.nanmean(boundary_incorrect_epochs,axis=0)
print(boundary_incorrect_epochs.shape)

(31, 63, 1101)
(14, 63, 1101)


##### Federico's original code
for electrode in electrode_name:
    electrode_idx = np.array(data_channels)==electrode

    plt.plot(boundary_correct_ERP[electrode_idx,:].T) # b_ERP[electrode_idx,:].squeeze()
    plt.plot(boundary_incorrect_ERP[electrode_idx,:].T)
    plt.legend(['B correct','B incorrect'])
    plt.xlabel('Time Rel. Stimulus Onset (ms)'); plt.ylabel('Voltage $\mu$V');
    plt.xticks([100,300,500,700,900],[0,200,400,600,800]); 
    plt.title(f'{condition_label}_{electrode}')

    figure_fname = os.path.join(data_folder, subject_code, 'eeg','analysis',
                               f'{subject_code}_boundary_source_{electrode}.pdf')
    # save the file 
    plt.savefig(figure_fname)
    plt.clf()

In [123]:
fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(20, 10)) 

# Flatten the axs array for easy iteration
axs = axs.flatten()

for ax, electrode in zip(axs, electrode_name):
    electrode_idx = np.array(data_channels) == electrode

    ax.plot(boundary_correct_ERP[electrode_idx,:].T)
    ax.plot(boundary_incorrect_ERP[electrode_idx,:].T)
    ax.legend(['B correct', 'B incorrect'])
    ax.set_xlabel('Time Rel. Stimulus Onset (ms)'); ax.set_ylabel('Voltage $\mu$V');
    ax.set_xticks([100,300,500,700,900]); ax.set_xticklabels([0,200,400,600,800]);
    ax.set_title(f'{condition_label}_{electrode}')
    
# Remove unused subplots when cols are not filled
for i in range(len(electrode_name), len(axs)):
    fig.delaxes(axs[i])

fig.suptitle('Comparison of ERPs for Correct and Incorrect Sources at Boundary Condition', fontsize=20)

figure_fname = os.path.join(data_folder, subject_code, 'eeg','analysis',
                            f'{subject_code}_boundary_source_All_Electrodes.pdf')
# save the file 
plt.savefig(figure_fname)


## Graphing CORRECT AND INCORRRECT TRIALS

In [30]:
#individual_ERP.squeeze().shape
